In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import time

In [12]:
chrome_driver_path = 'C:/Users/RobEnright/chromedriver.exe'

In [13]:
# Set up Chrome options (optional)
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--start-maximized")


In [14]:
# Set up ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [15]:
# Open a website
driver.get("https://ca.indeed.com/jobs?q=software+developer&l=Vancouver%2C+BC&from=searchOnHP")

# Print the title of the page
#print(driver.title)

In [16]:
# Locate the element using the provided XPath
first_tile = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "mosaic-provider-jobcards")))

first_tile

<selenium.webdriver.remote.webelement.WebElement (session="bbdf1cdd4e551b44770e1edbaa3ef1ff", element="f.EA67469EBB6B6ACD7AD3F1D958402498.d.CE09DC752CE5FE4D0686FEE299AAF112.e.22")>

In [17]:
# Find all <a> tags which contain hyperlinks
links = first_tile.find_elements(By.TAG_NAME, 'a')#for elem in elems:

# Extract the href attribute from each link
hyperlinks = [link.get_attribute("href") for link in links if link.get_attribute("href") is not None]


In [18]:
job_urls = []

# Print or process the hrefs
for hlink in hyperlinks:
    if 'clk?jk' in hlink:
        job_urls.append(hlink)

In [19]:
job_urls

['https://ca.indeed.com/rc/clk?jk=2d93e53c6290e161&bb=RuiZyQTc7wIeVaiOOObDnx1OODiaH2b6swRqJWvgzR4PI-9aTTiGBJPdtyrUhwxM9JMU7v0W2vhIYE3zbPjB6apc1i3z_Z35Q8NH9wosetRbzXIcNhLieA%3D%3D&xkcb=SoDS67M399ehcAxXYB0KbzkdCdPP&fccid=734cb5a01ee60f80&vjs=3',
 'https://ca.indeed.com/rc/clk?jk=bb01c33b9852ac89&bb=RuiZyQTc7wIeVaiOOObDn3_qO7oxbN2W_Cl0pro4PtbE3NILjhJ-1yoHFTn9Tu7rrVu-RExTZnpqHUH7_MPVtGhzwarx1-xwiDI9Kf8GmewhtPm9H4n1Gw%3D%3D&xkcb=SoD767M399ehcAxXYB0IbzkdCdPP&fccid=edae4285faf6c2f0&vjs=3',
 'https://ca.indeed.com/rc/clk?jk=9ce035772af131ad&bb=RuiZyQTc7wIeVaiOOObDn2pLnBOmhHFIgzOgNl-8-Z3UgsuW8a1Lr4tWGuI3yKCLrDwX9GpnwImz5P9koBqvY5gDfJRx9YF-jaCnWeteSphWAUWiRaF3KQ%3D%3D&xkcb=SoB167M399ehcAxXYB0PbzkdCdPP&fccid=734cb5a01ee60f80&vjs=3',
 'https://ca.indeed.com/rc/clk?jk=20d37f010faa56de&bb=RuiZyQTc7wIeVaiOOObDnze8TEs67SM5S8Ie99hy9R-At3VUMNsZ9ZyGNVVch2YoEmJdk80QMjt8hZGMtYiNmOQWWvKeJTNaQaHYKELy3OAk7w8Jvu-hRwO59mob4gpo&xkcb=SoDB67M399ehcAxXYB0ObzkdCdPP&fccid=01ecb1edcb54fdce&cmp=PhoneBox&ti=Back+End+Dev

In [20]:
driver.get(job_urls[0])
page_html = driver.page_source

In [ ]:
from bs4 import BeautifulSoup
import json

In [ ]:
soup = BeautifulSoup(page_html, 'html.parser')

In [13]:
# Extract job title
job_title = soup.find('h1', class_='jobsearch-JobInfoHeader-title').text.strip()
# Extract Company Details
company = soup.find('div', class_='css-hon9z8 eu4oa1w0')
# Pull Company Name
company_name = company.text
# Pull Company Hyperlink
company_link = company.find('a', href=True).get('href')

# Extract Job Details
job_details = soup.find('div', class_='js-match-insights-provider-kyg8or eu4oa1w0')
# Extract Pay Details
pay_section = job_details.find('div', {'aria-label': 'Pay'})
pay = pay_section.find('div', {'data-testid': True}).text if pay_section else "Pay not found"
# Extract Job type information
job_type_section = job_details.find('div', {'aria-label': 'Job type'})
job_type = [li.text for li in job_type_section.find_all('li', {'data-testid': True})] if job_type_section else "Job type not found"

# Extract job location
job_location = soup.find('div', class_='css-45str8 eu4oa1w0').text.strip()
# Extract job description
job_description = soup.find('div', class_='jobsearch-JobComponent-description css-16y4thd eu4oa1w0').text.strip()


In [14]:
job_dict = {
    'Title':job_title,
    'Company':company_name,
    'Company Link':company_link,
    'Pay Details':pay,
    'Job Type':job_type,
    'Location':job_location,
    'Description':job_description,
}

job_dict

{'Title': 'Software Engineer: Internship Opportunities, Vancouver, BC',
 'Company': 'Microsoft',
 'Company Link': 'https://ca.indeed.com/cmp/Microsoft?campaignid=mobvjcmp&from=mobviewjob&tk=1i5s5sgp3jdok803&fromjk=9ce035772af131ad',
 'Pay Details': '$5,400–$5,700 a month',
 'Job Type': 'Job type not found',
 'Location': 'Vancouver, BC',
 'Description': 'Come build community, explore your passions and do your best work at Microsoft with thousands of University interns from every corner of the world. This opportunity will allow you to bring your aspirations, talent, potential—and excitement for the journey ahead.\n \n\n\n As a Software Engineering Intern, you will work with teammates to solve problems and build innovative software solutions. You will apply your passion for customers and product quality as you provide technical guidance to Technical Program Managers and Product Managers. You will learn and adopt relevant new technologies, tools, methods, and processes to leverage in your 